In [ ]:
# Function (tools) calling

In [ ]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [ ]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

In [ ]:
# llm = get_llm("mistral_large_edenai")
# llm = get_llm("gpt_4omini_edenai")
# llm = get_llm("gpt_35_azure")
llm = get_llm("gpt_4_azure")
#llm = get_llm("llama32_3_ollama")
messages = []


@tool
def add(a: int, b: int) -> int:
    """Addition 2 integer numbers a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add]) # tool_choice="any"
prompt = def_prompt(
    "Use the provided functions  to compute what is 45 + 12; execute the function. Return 'I don't know' if there are not relevant function  "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

In [ ]:
debug(llm_with_tools)

In [ ]:
# Step 2 : Execute the function, and send result to LLM

In [ ]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

In [ ]:
debug(messages)

###  Other method : Use a tool calling agent

In [ ]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

llm = get_llm("gpt_4_azure")
#llm = get_llm("llama32_3_ollama")

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)"""
    print("exponentiate")
    return math.pow(x, y)


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant. Please use the provided tool"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
debug(r)

In [ ]:
## Call Eden.ai provided tools